In [17]:
import torch
from torch import nn, optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os
!pip freeze > requirements.txt
# device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
device = 'cpu'

hidden_size = 10
input_size = 370
output_size = 370
num_layers = 5
lr = 0.01
num_time_steps = 50

In [18]:
if not os.path.exists('data/train.pkl'):
    data = pd.read_csv(f'data/LD2011_2014.txt', sep=";")
    data.columns = ['Date'] + list(data.columns[1:])
    data[data.columns[1:]] = data[data.columns[1:]].apply(lambda x: [str(el).replace(',', '.') for el in x]).astype('float64')
    data['Date'] = data['Date'].astype('datetime64[ns]')
    train, validation, test = \
                np.split(data.sample(frac=1, random_state=42), 
                        [int(.7*len(data)), int(.9*len(data))])
                
    pickle.dump(train, open('data/train.pkl', 'wb'))
    pickle.dump(validation, open('data/validation.pkl', 'wb'))
    pickle.dump(test, open('data/test.pkl', 'wb'))
                
    print(f'Original Dataset: {len(data)}\nTrain Split: {len(train)}\nValidation Split: {len(validation)}\nTest Split: {len(test)}')
else:
    train = pickle.load(open('data/train.pkl', 'rb'))
    validation = pickle.load(open('data/validation.pkl', 'rb'))
    test = pickle.load(open('data/test.pkl', 'rb'))
    print(f'Train Split: {len(train)}\nValidation Split: {len(validation)}\nTest Split: {len(test)}')
    

Train Split: 98179
Validation Split: 28051
Test Split: 14026


In [22]:
class Net(nn.Module):
    def __init__(self, *args) -> None:
        super(Net, self).__init__()
        input_size, hidden_size, dropout, num_layers = args
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, dropout=dropout, num_layers=num_layers)
        print(self.rnn)

    def forward(self, x):
        print(x)
        
args = (input_size, hidden_size, 0.2, num_layers)
model = Net(*args)
model(train)

RNN(370, 10, num_layers=5, dropout=0.2)
                      Date     MT_001     MT_002    MT_003      MT_004  \
126107 2014-08-06 15:00:00  15.228426  33.428165  1.737619   85.365854   
86577  2013-06-20 20:30:00   5.076142  40.540541  2.606429  115.853659   
131762 2014-10-04 12:45:00   1.269036  32.005690  1.737619  107.723577   
75228  2013-02-22 15:15:00   3.807107  27.738265  2.606429   99.593496   
43795  2012-04-01 05:00:00  17.766497  19.203414  1.737619   65.040650   
...                    ...        ...        ...       ...         ...   
96633  2013-10-03 14:30:00  19.035533  28.449502  1.737619   69.105691   
16026  2011-06-16 22:45:00   0.000000   0.000000  0.000000    0.000000   
125665 2014-08-02 00:30:00  17.766497  30.583215  1.737619  144.308943   
125655 2014-08-01 22:00:00  16.497462  30.583215  1.737619  132.113821   
95684  2013-09-23 17:15:00  13.959391  32.005690  1.737619  113.821138   

           MT_005      MT_006     MT_007      MT_008     MT_009  ...  \